In [10]:
import pandas as pd

# ------------------------- 1️⃣ Load UHI Dataset -------------------------
print("\n🔹 Loading UHI Dataset...")
uhi_path = "../data/processed/UHI_with_LST_Sentinel_Final.csv"
uhi_df = pd.read_csv(uhi_path)

# Convert datetime column
uhi_df["datetime"] = pd.to_datetime(uhi_df["datetime"], dayfirst=True, errors="coerce")

print(f"✅ UHI Dataset Loaded. Shape: {uhi_df.shape}")
print(f"🔹 UHI Columns: {uhi_df.columns}\n")


# ------------------------- 2️⃣ Load Weather Dataset -------------------------
print("\n🔹 Loading Weather Dataset...")
weather_path = "../data/raw/NY_Mesonet_Weather.xlsx"

# Load all sheets
weather_sheets = pd.ExcelFile(weather_path).sheet_names
print(f"📌 Available Weather Sheets: {weather_sheets}")

weather_dfs = []
for sheet in ["Bronx", "Manhattan"]:
    df = pd.read_excel(weather_path, sheet_name=sheet)
    
    # Fix datetime column issue
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    df.rename(columns={"date_/_time": "datetime"}, inplace=True)

    if "datetime" not in df.columns:
        print(f"⚠️ 'datetime' column not found in {sheet}, skipping merge.")
        continue  

    # Remove timezone and parse datetime correctly
    df["datetime"] = df["datetime"].astype(str).str.replace(" EDT", "", regex=False)
    df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

    weather_dfs.append(df)

# Merge Bronx & Manhattan Weather data
weather_df = pd.concat(weather_dfs, ignore_index=True)

# Remove duplicates by averaging over timestamps
weather_df = weather_df.groupby("datetime").mean().reset_index()

# 🔹 Fix: Set datetime as index before interpolation
weather_df.set_index("datetime", inplace=True)
weather_df.interpolate(method="time", inplace=True)
weather_df.reset_index(inplace=True)  # Restore datetime as a column

print(f"✅ Weather Data Processed. Shape: {weather_df.shape}")
print(f"🔹 Weather Columns: {weather_df.columns}\n")


# ------------------------- 3️⃣ Merge UHI with Weather Data -------------------------
print("\n🔹 Merging UHI with Weather Data...")
merged_df = pd.merge(uhi_df, weather_df, on="datetime", how="left")

# 🔹 Fill remaining missing values with forward fill method
# Handle missing values using forward fill, backward fill, and interpolation
merged_df.set_index("datetime", inplace=True)  # Set datetime as index
merged_df = merged_df.interpolate(method="time")  # Time-based interpolation
merged_df.fillna(method="ffill", inplace=True)  # Forward fill
merged_df.fillna(method="bfill", inplace=True)  # Backward fill
merged_df.reset_index(inplace=True)  # Restore datetime column

print(f"✅ Merged Weather Data. Shape: {merged_df.shape}")

# Save the final merged dataset
output_path = "../data/processed/UHI_Weather_Merged.csv"
merged_df.to_csv(output_path, index=False)
print(f"✅ Final dataset saved: {output_path}")


# ------------------------- 4️⃣ Summary Statistics -------------------------
print("\n📌 Final Merged Dataset Summary:")
print(merged_df.info())
print(merged_df.head())


🔹 Loading UHI Dataset...
✅ UHI Dataset Loaded. Shape: (11229, 9)
🔹 UHI Columns: Index(['Longitude', 'Latitude', 'datetime', 'UHI Index', 'Land_Surface_Temp',
       'Band1', 'Band2', 'Band3', 'Band4'],
      dtype='object')


🔹 Loading Weather Dataset...
📌 Available Weather Sheets: ['Summary', 'Terms', 'Location', 'Bronx', 'Manhattan']
✅ Weather Data Processed. Shape: (169, 6)
🔹 Weather Columns: Index(['datetime', 'air_temp_at_surface_[degc]', 'relative_humidity_[percent]',
       'avg_wind_speed_[m/s]', 'wind_direction_[degrees]',
       'solar_flux_[w/m^2]'],
      dtype='object')


🔹 Merging UHI with Weather Data...
✅ Merged Weather Data. Shape: (11229, 14)
✅ Final dataset saved: ../data/processed/UHI_Weather_Merged.csv

📌 Final Merged Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11229 entries, 0 to 11228
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----    

/var/folders/y0/slhcnc5d5ylcv5sf0glxtrvr0000gn/T/ipykernel_39340/1279355451.py:37: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")
/var/folders/y0/slhcnc5d5ylcv5sf0glxtrvr0000gn/T/ipykernel_39340/1279355451.py:37: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")
/var/folders/y0/slhcnc5d5ylcv5sf0glxtrvr0000gn/T/ipykernel_39340/1279355451.py:64: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method="ffill", inplace=True)  # Forward fill
/var/folders/y0/slhcnc5d5ylcv5sf0glxtrvr0000gn/T/ipykernel_39340/127935545

In [11]:
merged_df.isnull().sum()
merged_df.describe()

,datetime,Longitude,Latitude,UHI Index,Land_Surface_Temp,Band1,Band2,Band3,Band4,air_temp_at_surface_[degc],relative_humidity_[percent],avg_wind_speed_[m/s],wind_direction_[degrees],solar_flux_[w/m^2]
count,11229,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000
mean,2021-07-24 15:34:29.056906240,-73.933927,40.808800,1.000001,36.901146,1577.434233,1639.084691,2150.186036,2225.908273,27.102467,46.746034,3.064635,159.795342,465.091290
min,2021-07-24 15:01:00,-73.994457,40.758792,0.956122,27.702375,265.000000,167.000000,245.000000,210.000000,26.850000,45.400000,2.100000,118.500000,217.500000
25%,2021-07-24 15:22:00,-73.955703,40.790905,0.988577,35.717631,883.000000,828.000000,1484.000000,1474.000000,26.980000,45.650000,2.950000,138.500000,391.400000
50%,2021-07-24 15:36:00,-73.932968,40.810688,1.000237,37.177126,1173.000000,1276.000000,1884.000000,1928.000000,27.100000,47.000000,3.040000,164.100000,450.300000
75%,2021-07-24 15:48:00,-73.909647,40.824515,1.011176,38.431539,1554.000000,1798.000000,2355.000000,2586.000000,27.250000,47.650000,3.300000,179.000000,557.600000
max,2021-07-24 15:59:00,-73.879458,40.859497,1.046036,46.781762,12280.000000,12736.000000,12682.000000,12992.000000,27.300000,48.050000,4.150000,186.000000,618.000000
std,NaN,0.028253,0.023171,0.016238,2.460982,1510.100390,1456.841224,1380.281893,1425.736087,0.148587,0.917194,0.403778,20.931707,104.127503
